In [1]:
import pandas as pd
train, test = pd.read_csv('train.csv') , pd.read_csv('test.csv')

print(train.head())


   Annual_Income  Num_of_Delayed_Payment  Monthly_Inhand_Salary Credit_Score
0      22380.180                    16.0            1385.264539         Good
1      99166.770                     7.0            8305.897500         Good
2      10496.255                    17.0            1141.687917         Poor
3      44438.410                     2.0            3974.200833         Good
4      15330.305                    19.0            1135.525417         Good


In [2]:
# do not change this code
import numpy as np
import pandas as pd 

x_columns_names =  ['Num_of_Delayed_Payment','Annual_Income','Monthly_Inhand_Salary']
y_column_name = 'Credit_Score'
label_to_index = {'Poor':0,'Good': 1}
index_to_label = {v:k for k,v in label_to_index.items()}

def prepare_dataset(data_frame: pd.DataFrame): 
     x_columns = [list(data_frame[col]) for col in x_columns_names]
     return np.array(list(zip(*x_columns)),dtype=np.float64), np.array(list(data_frame[y_column_name].apply(lambda x: label_to_index[x])),dtype=np.int8)

In [3]:
# do not change this code
trainX, trainY = prepare_dataset(train)
testX, testY = prepare_dataset(test)

In [4]:
list(set(trainY))

[0, 1]

In [5]:
trainX.shape, trainY.shape

((8000, 3), (8000,))

In [6]:
testX.shape, testY.shape

((2000, 3), (2000,))

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

def train_model(trainX, trainY):
    mlp_model = MLPClassifier(random_state=42)
    mlp_model.fit(trainX, trainY)

    nb_model = GaussianNB()
    nb_model.fit(trainX, trainY)

    return mlp_model, nb_model

mlp_model, nb_model = train_model(trainX, trainY)

In [13]:
def generate_predictions(model, X, Y):
    predictions = []
    for x, y in zip(X, Y):
        train_predictions = model.predict(np.array([x]))
        predictions.append(train_predictions)
        
    return predictions

train_predictions_mlp = generate_predictions(mlp_model, trainX, trainY)
train_predictions_nb = generate_predictions(nb_model, trainX, trainY)

test_predictions_mlp = generate_predictions(mlp_model, testX, testY)
test_predictions_nb = generate_predictions(nb_model, testX, testY)


In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def calculate_metrics(y_pred, y_truth):
    accuracy = accuracy_score(y_truth, y_pred)
    
    cm = confusion_matrix(y_truth, y_pred)
    
    error_rates = cm.diagonal() / cm.sum(axis=1)
    
    most_misclassified_class = np.argmax(error_rates)
    
    report = classification_report(y_truth, y_pred, output_dict=True, zero_division=0)
    
    return {
        'accuracy': accuracy,
        'error_rates': error_rates,
        'most_misclassified_class': most_misclassified_class,
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall'],
        'f1-score': report['weighted avg']['f1-score']
    }

mlp_metrics_test = calculate_metrics(test_predictions_mlp, testY)

nb_metrics_test = calculate_metrics(test_predictions_nb, testY)

mlp_metrics_train = calculate_metrics(train_predictions_mlp, trainY)

nb_metrics_train = calculate_metrics(train_predictions_nb, trainY)


In [15]:
print(nb_metrics_train)

{'accuracy': 0.68275, 'error_rates': array([0.91365299, 0.31351527]), 'most_misclassified_class': 0, 'precision': 0.6856853845818958, 'recall': 0.68275, 'f1-score': 0.6460406267438146}
